#### Library

In [1]:
import numpy as np
import pandas as pd

### Dataset

In [2]:
data = pd.DataFrame([
    {'host': 1, 'country': 'fr', 'year': 2010, 'month': 1, 'clicks': 123, 'users': 4},
    {'host': 1, 'country': 'fr', 'year': 2010, 'month': 2, 'clicks': 134, 'users': 5},
    {'host': 1, 'country': 'fr', 'year': 2010, 'month': 3, 'clicks': 341, 'users': 2},
    {'host': 1, 'country': 'es', 'year': 2010, 'month': 1, 'clicks': 113, 'users': 4},
    {'host': 1, 'country': 'es', 'year': 2010, 'month': 2, 'clicks': 234, 'users': 5},
    {'host': 1, 'country': 'es', 'year': 2010, 'month': 3, 'clicks': 421, 'users': 2},
    {'host': 1, 'country': 'es', 'year': 2010, 'month': 4, 'clicks': 22, 'users': 3},
    {'host': 2, 'country': 'es', 'year': 2010, 'month': 1, 'clicks': 111, 'users': 2},
    {'host': 2, 'country': 'es', 'year': 2010, 'month': 2, 'clicks': 2, 'users': 4},
    {'host': 3, 'country': 'es', 'year': 2010, 'month': 3, 'clicks': 34, 'users': 2},
    {'host': 3, 'country': 'es', 'year': 2010, 'month': 4, 'clicks': 1, 'users': 1}
])

In [3]:
data

,host,country,year,month,clicks,users
0,1,fr,2010,1,123,4
1,1,fr,2010,2,134,5
2,1,fr,2010,3,341,2
3,1,es,2010,1,113,4
4,1,es,2010,2,234,5
5,1,es,2010,3,421,2
6,1,es,2010,4,22,3
7,2,es,2010,1,111,2
8,2,es,2010,2,2,4
9,3,es,2010,3,34,2


#### Pivot table

In [4]:
pd.pivot_table(data,
   index=['host'],
   values=['users', 'clicks'],
   columns=['year', 'month'],
   fill_value=''
  )

clicks                     users               
year    2010                      2010               
month      1      2      3     4     1    2    3    4
host                                                 
1      118.0  184.0  381.0  22.0   4.0  5.0  2.0  3.0
2      111.0    2.0                2.0  4.0          
3                     34.0   1.0             2.0  1.0

In [5]:
pd.pivot_table(data,
               index=['host'],
               values=['users', 'clicks'],
               columns=['year', 'month'],
               fill_value='',
               aggfunc=np.sum,
               margins=True,
               margins_name='Total'
              )

clicks                           users                      
year    2010                     Total  2010                 Total
month      1      2      3     4           1     2    3    4      
host                                                              
1      236.0  368.0  762.0  22.0  1388   8.0  10.0  4.0  3.0    25
2      111.0    2.0                113   2.0   4.0               6
3                     34.0   1.0    35              2.0  1.0     3
Total    347    370    796    23  1536    10    14    6    4    34

In [6]:
data.groupby(['host', 'country'])[['users', 'clicks']].sum()

users  clicks
host country               
1    es          14     790
     fr          11     598
2    es           6     113
3    es           3      35

In [7]:
data.groupby(['host', 'country']).agg({'users':sum, 'clicks':sum})

users  clicks
host country               
1    es          14     790
     fr          11     598
2    es           6     113
3    es           3      35

In [8]:
data.groupby(['host', 'country'], as_index= False).agg(total_users=('users', sum), total_clicks=('clicks', sum))

,host,country,total_users,total_clicks
0,1,es,14,790
1,1,fr,11,598
2,2,es,6,113
3,3,es,3,35


In [9]:
out = data.groupby('host').apply(lambda sub: sub.pivot_table(
    index=['host', 'country'],
    values=['users', 'clicks'],
    columns=['year', 'month'],
    aggfunc=np.sum,
    margins=True,
    margins_name='SubTotal',
))

out.loc[('', 'Max', '')] = out.max()
out.loc[('', 'Min', '')] = out.min()
out.loc[('', 'Total', '')] = out.sum()


In [10]:
out

clicks                                users        \
year                     2010                       SubTotal  2010         
month                       1       2       3     4              1     2   
host host     country                                                      
1    1        es        113.0   234.0   421.0  22.0    790.0   4.0   5.0   
              fr        123.0   134.0   341.0   NaN    598.0   4.0   5.0   
     SubTotal           236.0   368.0   762.0  22.0   1388.0   8.0  10.0   
2    2        es        111.0     2.0     NaN   NaN    113.0   2.0   4.0   
     SubTotal           111.0     2.0     NaN   NaN    113.0   2.0   4.0   
3    3        es          NaN     NaN    34.0   1.0     35.0   NaN   NaN   
     SubTotal             NaN     NaN    34.0   1.0     35.0   NaN   NaN   
     Max                236.0   368.0   762.0  22.0   1388.0   8.0  10.0   
     Min                111.0     2.0    34.0   1.0     35.0   2.0   4.0   
     Total             1041.0  1110.0  2388.0  69.0   4495.0  30.0  42.0   

                                            
year                              SubTotal  
month                     3     4           
host host     country                       
1    1        es        2.0   3.0     14.0  
              fr        2.0   NaN     11.0  
     SubTotal           4.0   3.0     25.0  
2    2        es        NaN   NaN      6.0  
     SubTotal           NaN   NaN      6.0  
3    3        es        2.0   1.0      3.0  
     SubTotal           2.0   1.0      3.0  
     Max                4.0   3.0     25.0  
     Min                2.0   1.0      3.0  
     Total             18.0  12.0     96.0

In [11]:
out.index = out.index.droplevel(0)

In [12]:
out.fillna('', inplace=True)

In [13]:
out

clicks                                users              \
year                2010                       SubTotal  2010               
month                  1       2       3     4              1     2     3   
host     country                                                            
1        es        113.0   234.0   421.0  22.0    790.0   4.0   5.0   2.0   
         fr        123.0   134.0   341.0          598.0   4.0   5.0   2.0   
SubTotal           236.0   368.0   762.0  22.0   1388.0   8.0  10.0   4.0   
2        es        111.0     2.0                  113.0   2.0   4.0         
SubTotal           111.0     2.0                  113.0   2.0   4.0         
3        es                         34.0   1.0     35.0               2.0   
SubTotal                            34.0   1.0     35.0               2.0   
Max                236.0   368.0   762.0  22.0   1388.0   8.0  10.0   4.0   
Min                111.0     2.0    34.0   1.0     35.0   2.0   4.0   2.0   
Total             1041.0  1110.0  2388.0  69.0   4495.0  30.0  42.0  18.0   

                                 
year                   SubTotal  
month                4           
host     country                 
1        es        3.0     14.0  
         fr                11.0  
SubTotal           3.0     25.0  
2        es                 6.0  
SubTotal                    6.0  
3        es        1.0      3.0  
SubTotal           1.0      3.0  
Max                3.0     25.0  
Min                1.0      3.0  
Total             12.0     96.0